In [1]:
import pandas as pd
import geopandas as gdp
import fiona 
import shapely as sh
import folium as web
import sys
from folium import plugins
import branca
import warnings
warnings.filterwarnings("ignore")

In [2]:
#### total data
Senate_gdf = gdp.read_file("/Users/michaelremington/Documents/OCLT/SenateGeoFile.geojson")
House_gdf = gdp.read_file("/Users/michaelremington/Documents/OCLT/HouseGeoFIle.geojson")
counties_gdf = gdp.read_file('/Users/michaelremington/Documents/OCLT/Oregon_County_Boundaries')
#### Label Sets
senate_labels=Senate_gdf[['NAMELSAD','INTPTLAT','INTPTLON']]
house_labels=House_gdf[['NAMELSAD','INTPTLAT','INTPTLON']]
senate_labels['District Number']=pd.to_numeric(senate_labels["NAMELSAD"].apply(lambda x: x[len(x)-2:len(x)]))
house_labels['District Number']=pd.to_numeric(house_labels["NAMELSAD"].apply(lambda x: x[len(x)-2:len(x)]))
#### Pop Up Info
senate_info=pd.read_excel('/Users/michaelremington/Documents/OCLT/2019 Master List - land trust and leg.xlsx',
                          sheet_name='Senate', header=0)
house_info=pd.read_excel('/Users/michaelremington/Documents/OCLT/2019 Master List - land trust and leg.xlsx',
                          sheet_name='House', header=0)
senate_info['Full Name'] = senate_info['First Name'] + ' ' + senate_info['Last Name']
house_info['Full Name'] = house_info['First Name'] + ' ' + house_info['Last Name']
#### Data Clean Up 
senate_labels['INTPTLAT']=pd.to_numeric(senate_labels['INTPTLAT'])
senate_labels['INTPTLON']=pd.to_numeric(senate_labels['INTPTLON'])
house_labels['INTPTLAT']=pd.to_numeric(house_labels['INTPTLAT'])
house_labels['INTPTLON']=pd.to_numeric(house_labels['INTPTLON'])
#### Final info
county_labels=counties_gdf[['geometry','instName']]
county_labels['lng']=counties_gdf['geometry'].centroid.x
county_labels['lat']=counties_gdf['geometry'].centroid.y
county_labels=county_labels[['lng','lat','instName']]
senate_labels=senate_labels.merge(senate_info, how='inner', on='District Number')
house_labels=house_labels.merge(house_info, how='inner', on='District Number')

In [3]:
def senate_fancy_html(row):
    i = row
    
    District = senate_labels['NAMELSAD'].iloc[i]                             
    Senator = senate_labels['Full Name'].iloc[i]                           
    Phone = senate_labels['Capitol Phone'].iloc[i]
    Email = senate_labels['Member Email'].iloc[i]                                           
    Web_Address = senate_labels['Web Address'].iloc[i]                               
    Capitol_Address = senate_labels['Capitol Address'].iloc[i]                             

    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(District) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Senator</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Senator) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Phone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Phone) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Email</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Email) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Web Address</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Web_Address) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Capitol Address</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Capitol_Address) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

def house_fancy_html(row):
    i = row
    
    District = house_labels['NAMELSAD'].iloc[i]                             
    House_Reprsentitve = house_labels['Full Name'].iloc[i]                           
    Phone = house_labels['Capitol Phone'].iloc[i]
    Email = house_labels['Member email'].iloc[i]                                           
    Web_Address = house_labels['Web Address'].iloc[i]                               
    Capitol_Address = house_labels['MemberContact'].iloc[i]                             

    left_col_colour = "#f45042"
    right_col_colour = "#febdaf"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(District) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">House Representative</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(House_Reprsentitve) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Phone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Phone) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Email</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Email) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Web Address</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Web_Address) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Capitol Address</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Capitol_Address) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [4]:
zoom = 5
centerpoint=[43.8041, -120.5542] ### Orients the map when you bring it up
m= web.Map(centerpoint,tiles="stamenterrain",zoom_start=zoom) ### There are a couple of different tiles https://python-graph-gallery.com/288-map-background-with-folium/
###### Folium GeoJson makes layer
#senate=web.GeoJson(Senate_gdf,name='Senate Shape').add_to(m)

###### This is another way to make a shaded and interactive layer not sure differnece between this and other. I think Cloropleth is more customizable
m.choropleth(
        geo_data= Senate_gdf,
        name = 'Senate Shape',
        #data = House_gdf,
        #columns=['NAMELSAD'],
        #key_on='GEOID',
        fill_color='blue',
        fill_opacity=0,
        line_opacity=1,
        line_color='blue',
        legend_name='Senate Shape'
        )
m.choropleth(
        geo_data= House_gdf,
        name = 'House Shape',
        #data = House_gdf,
        #columns=['NAMELSAD'],
        #key_on='GEOID',
        fill_color='red',
        fill_opacity=0,
        line_opacity=1,
        line_color='red',
        legend_name='House Districts'
        )

m.choropleth(
        geo_data= counties_gdf,
        name = 'Oregon Counties',
        #data = House_gdf,
        #columns=['NAMELSAD'],
        #key_on='GEOID',
        fill_color='black',
        fill_opacity=.05,
        line_opacity=.8,
        line_color='black',
        legend_name='Counties'
        )


##### Marker clusters collapse markers based on zoom

mcg = web.plugins.MarkerCluster(control=False) # makes clustering filterable
m.add_child(mcg)

mcg2 = web.plugins.MarkerCluster(control=False) # makes clustering filterable
m.add_child(mcg2)

mcg3 = web.plugins.MarkerCluster(control=False) # makes clustering filterable
m.add_child(mcg3)

##### Marker Groups makes markers filterable

## To make parent group
##fg = web.FeatureGroup(name='Markers')
#m.add_child(fg) ## add_child is a coinvienent way to add stuff to the map

Senate_Markers = plugins.FeatureGroupSubGroup(mcg, 'Senate Markers')
m.add_child(Senate_Markers)

House_Markers = plugins.FeatureGroupSubGroup(mcg2, 'House Markers')
m.add_child(House_Markers)

County_Markers = plugins.FeatureGroupSubGroup(mcg3, 'County Markers')
m.add_child(County_Markers)


##### How to make Markers
for x,index in senate_labels.iterrows():
    html = senate_fancy_html(x) 
    iframe = branca.element.IFrame(html=html,width=400,height=200)
    popup = web.Popup(iframe,parse_html=True)

    web.Marker(location=([senate_labels['INTPTLAT'].loc[x],
                        senate_labels['INTPTLON'].loc[x]]),                        
                        #tooltip=senate_labels['NAMELSAD'].loc[x],
                        popup=popup).add_to(Senate_Markers)
### Add popup= to add text when you click on marker
for x,index in house_labels.iterrows():
    html = house_fancy_html(x)
    iframe = branca.element.IFrame(html=html,width=400,height=200)
    popup = web.Popup(iframe,parse_html=True)
    web.Marker(location=([house_labels['INTPTLAT'].loc[x],
                        house_labels['INTPTLON'].loc[x]]),                        
                        tooltip=house_labels['NAMELSAD'].loc[x],
                        popup=popup,
                        icon = web.Icon(color='red', icon='info-sign')).add_to(House_Markers)
for x,index in county_labels.iterrows():
    web.Marker(location=([county_labels['lat'].loc[x],
                        county_labels['lng'].loc[x]]),                        
                        tooltip=county_labels['instName'].loc[x],
                        icon = web.Icon(color='black', icon='star')).add_to(County_Markers)
    
    
######### Add Search Bars 
'''
SenateSearch = plugins.Search(
    layer=Senate_Markers,
    geom_type='Point',
    placeholder='Search for a Oregon Senator',
    collapsed=False,
    search_label='Senate Marker',
    weight=3
).add_to(m)

Housesearch = plugins.Search(
    layer=House_Markers,
    geom_type='Point',
    placeholder='Search for House Representative',
    collapsed=True,
    search_label='House Marker'
).add_to(m)'''
web.LayerControl().add_to(m) ### This makes layers filterable

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m) #### allows you to go in and out of full screen
m.save('County_map.html') ### Makes HTML file
    
